In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
import glob
from pathlib import Path
import os
from PIL import Image 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv("SCORNdata.csv",header=None)

In [2]:
len(data)

6242

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8,9
0,55.98,8.1,5.77,11.99,2.08,1.75,1.19,0.039,98.81,670
1,56.00,8.3,5.79,12.01,2.07,1.93,1.19,0.038,99.04,850
2,56.00,8.8,5.67,11.98,2.11,1.78,1.18,0.037,98.68,840
3,55.99,8.0,5.69,12.05,2.12,1.74,1.19,0.039,98.80,850
4,55.95,8.0,5.63,11.97,2.13,1.85,1.18,0.040,98.72,850


In [4]:
#Data Loading
data = pd.read_csv("SCORNdata.csv",header=None)
data.head()

# # print(dataX)
# print(dataX.shape)
# # print(dataY)
# print(dataY.shape)

## validation
D=data.to_numpy();
k=5;
np.random.shuffle(D)
dataset = np.array_split(D, k)
train_set = dataset.copy()
test_set = train_set.pop(3)
train_set = np.vstack(train_set)
train_set=np.array(train_set)
test_set=np.array(test_set)
trainX=train_set[:,9:10]
#print(trainX.shape)
trainY=train_set[:,0:9]
#print(trainY.shape)
testX=test_set[:,9:10]
#print(testX.shape)
testY=test_set[:,0:9]
#print(testY.shape)


print(testX.shape)
print(testY.shape)

(1248, 1)
(1248, 9)


In [5]:
trainX =trainX.reshape((trainX.shape[0],1, 1,1))
trainX=torch.from_numpy(trainX).to(torch.float32)
trainY=torch.Tensor(trainY)


testX =testX.reshape((testX.shape[0],1, 1,1))
testX=torch.from_numpy(testX).to(torch.float32)
testY=torch.Tensor(testY)

#print(trainX.shape)
#print(trainY.shape)
#print(testX.shape)
#print(testY.shape)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,12,kernel_size=1)
        self.relu=nn.ReLU()
        self.BN= nn.BatchNorm2d(12)
        self.maxpool=nn.MaxPool2d(1,2)
        self.avgpool =nn.AvgPool2d(1,2)
        
        #self.conv2=nn.Conv2d(in_channels=8,
        #                   out_channels=15,
        #                     kernel_size=3)
       # self.conv3=nn.Conv2d(in_channels=10,
      #                      out_channels=20,
       #                      kernel_size=3)

        self.fc=nn.Linear(in_features=12,out_features=9)
        self.dropout=nn.Dropout(p=0.7)
   
    def forward(self,x):
        x=self.conv1(x)
        x=self.relu(x)
        x = self.BN(x)
        x=self.avgpool(x)
        x = self.BN(x)
        x=self.maxpool(x)
        

        

        x=self.dropout(x)

        
       # x.view(x.size(0), self.num_classes)
        #print(x.size(0))
        x = x.view(x.size(0),12) 
        x=self.fc(x)
              
        return x

model=Net()
print(model)

Net(
  (conv1): Conv2d(1, 12, kernel_size=(1, 1), stride=(1, 1))
  (relu): ReLU()
  (BN): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avgpool): AvgPool2d(kernel_size=1, stride=2, padding=0)
  (fc): Linear(in_features=12, out_features=9, bias=True)
  (dropout): Dropout(p=0.7, inplace=False)
)


In [7]:
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import r2_score # R square
from math import sqrt

criterion=nn.MSELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.05)
losses=[]
epochs=20000
model.train()

for epoch in range(epochs):
       
        y_pred=model(trainX)

        loss=criterion(trainY,y_pred)
        print(loss.item())
        losses.append(loss.item)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                
        yhatt = model(testX).detach().numpy()
        ytest = testY.detach().numpy()
    

1447.8822021484375
1408.541259765625
1346.416015625
1177.9307861328125
735.86669921875
509.131103515625
497.6384582519531
407.0288391113281
511.6167297363281
378.8399353027344
490.081787109375
352.24420166015625
449.3083190917969
320.4909973144531
435.65771484375
302.5419616699219
393.7514343261719
303.25616455078125
412.0744323730469
262.8078918457031
320.04827880859375
268.2403869628906
349.9037170410156
274.4128112792969
352.74005126953125
264.0993347167969
336.65673828125
272.01861572265625
372.9970703125
256.12939453125
349.61883544921875
248.489501953125
306.0146179199219
250.7366180419922
320.7817687988281
243.442626953125
296.9956970214844
236.51475524902344
298.12982177734375
231.7935333251953
274.5384521484375
231.29864501953125
279.2456359863281
232.70654296875
277.4814758300781
224.9553680419922
278.2807312011719
218.96253967285156
258.5523681640625
218.3805389404297
274.6837463378906
222.56414794921875
256.94891357421875
211.16815185546875
249.04119873046875
203.5316162109

38.199771881103516
38.12377166748047
37.96493148803711
37.651702880859375
35.946044921875
37.10232162475586
37.67501449584961
35.9788818359375
36.201168060302734
36.489463806152344
36.71110916137695
35.60972213745117
36.932403564453125
36.045650482177734
36.69792175292969
35.70414352416992
36.88693618774414
35.15061569213867
34.856266021728516
35.06479263305664
36.15962600708008
33.7557258605957
34.83600997924805
34.16825485229492
34.92686080932617
34.75443649291992
34.22520446777344
34.132728576660156
34.20745849609375
34.46548080444336
34.98017501831055
34.204856872558594
33.4258918762207
33.59425735473633
32.99407958984375
33.85301971435547
33.38783645629883
32.30217742919922
32.33308410644531
34.109230041503906
32.989994049072266
31.82643699645996
32.68189239501953
32.753318786621094
32.396968841552734
32.855403900146484
31.612529754638672
31.678674697875977
32.0709342956543
32.956817626953125
32.28835678100586
30.703571319580078
30.976707458496094
31.831388473510742
31.66854476928

7.369801998138428
7.156789302825928
7.509954452514648
7.131526470184326
7.322860240936279
7.340451717376709
7.244133949279785
7.141881942749023
7.274568557739258
6.852120399475098
7.186340808868408
7.045478820800781
6.965633869171143
7.130941390991211
6.8363494873046875
7.1948089599609375
7.193393230438232
7.085997581481934
6.792172908782959
7.01550817489624
6.983277797698975
6.899845123291016
6.9160590171813965
6.872955322265625
6.8284101486206055
6.753671169281006
6.458616256713867
6.754434585571289
6.603436470031738
6.537965774536133
6.703223705291748
6.67969274520874
6.370830535888672
6.627786636352539
6.512897491455078
6.5247602462768555
6.652575969696045
6.363762378692627
6.555576324462891
6.609475135803223
6.52409553527832
6.507201671600342
6.3937177658081055
6.257052898406982
6.488719940185547
6.376772403717041
6.167025566101074
6.353184223175049
6.363979816436768
6.130557537078857
5.98471212387085
6.460402965545654
5.982110977172852
6.1586103439331055
6.133588790893555
6.06964

1.5235419273376465
1.5420405864715576
1.5221543312072754
1.5381770133972168
1.502152442932129
1.4746087789535522
1.489157795906067
1.4833964109420776
1.4854203462600708
1.518022894859314
1.4855302572250366
1.5287814140319824
1.4844566583633423
1.4667302370071411
1.4678465127944946
1.4721379280090332
1.4860235452651978
1.4360560178756714
1.3828996419906616
1.4183787107467651
1.4119971990585327
1.4175114631652832
1.4413409233093262
1.447766900062561
1.404841423034668
1.456850528717041
1.3988927602767944
1.436980128288269
1.3931103944778442
1.4328869581222534
1.3748759031295776
1.3477007150650024
1.398481011390686
1.3793225288391113
1.365752935409546
1.42308509349823
1.3515452146530151
1.4162251949310303
1.3312252759933472
1.3531728982925415
1.3788374662399292
1.3787604570388794
1.3262940645217896
1.3229976892471313
1.3275092840194702
1.3437200784683228
1.2917929887771606
1.3392378091812134
1.2835031747817993
1.3598464727401733
1.3003239631652832
1.3244783878326416
1.3105310201644897
1.31

0.5215749144554138
0.5030787587165833
0.5076256394386292
0.5158632397651672
0.5211952924728394
0.5154244899749756
0.5042516589164734
0.510377824306488
0.5016529560089111
0.5035140514373779
0.5015674829483032
0.5067717432975769
0.5069661736488342
0.4948263466358185
0.49480104446411133
0.5008660554885864
0.4961879551410675
0.4996350109577179
0.5063665509223938
0.49157434701919556
0.4951079189777374
0.48864996433258057
0.4996936321258545
0.4929269850254059
0.49675512313842773
0.489472359418869
0.4984898567199707
0.48849162459373474
0.48853906989097595
0.4907255172729492
0.48273152112960815
0.4860650897026062
0.476360946893692
0.4740940034389496
0.4889281094074249
0.488870769739151
0.48475661873817444
0.4791203439235687
0.48880958557128906
0.48578107357025146
0.4778824746608734
0.47412553429603577
0.47440305352211
0.48107224702835083
0.48062899708747864
0.4837704002857208
0.4807066321372986
0.46383169293403625
0.46812868118286133
0.4694196283817291
0.47304603457450867
0.47754910588264465
0

...

0.23370859026908875
0.2338222712278366
0.23361514508724213
0.23369692265987396
0.23387610912322998
0.23359796404838562
0.23396404087543488
0.2333444058895111
0.23355916142463684
0.2333020567893982
0.23359555006027222
0.23275867104530334
0.2341507226228714
0.23409725725650787
0.2328561693429947
0.2339116334915161
0.23394496738910675
0.23366352915763855
0.23385822772979736
0.2339434027671814
0.23370814323425293
0.23352345824241638
0.23355799913406372
0.23408016562461853
0.23405121266841888
0.23353241384029388
0.23394638299942017
0.23411917686462402
0.23422513902187347
0.23365958034992218
0.23322737216949463
0.23334883153438568
0.23369227349758148
0.23405371606349945
0.233557790517807
0.23382511734962463
0.23334679007530212
0.2336052805185318
0.23355333507061005
0.23342333734035492
0.2334512323141098
0.2333548665046692
0.23390242457389832
0.2335261106491089
0.23397091031074524
0.23365437984466553
0.23388320207595825
0.23362410068511963
0.2334146648645401
0.23413647711277008
0.233529642224

0.2339211106300354
0.23349443078041077
0.2339949905872345
0.23402227461338043
0.23347552120685577
0.23339112102985382
0.23380888998508453
0.23327629268169403
0.2335546314716339
0.23395420610904694
0.23373401165008545
0.23330940306186676
0.23328830301761627
0.23332558572292328
0.23414814472198486
0.23345497250556946
0.23365309834480286
0.23369406163692474
0.2335154116153717
0.23400487005710602
0.2337827980518341
0.23363018035888672
0.23325477540493011
0.23401063680648804
0.23357605934143066
0.2338772416114807
0.23406755924224854
0.2341242879629135
0.23365972936153412
0.23369312286376953
0.23316432535648346
0.23374022543430328
0.23356600105762482
0.23350554704666138
0.2333104908466339
0.23341630399227142
0.2337961196899414
0.2340654879808426
0.23373717069625854
0.23358909785747528
0.23370370268821716
0.23354993760585785
0.23389074206352234
0.23354646563529968
0.23394343256950378
0.2336200326681137
0.23340995609760284
0.23367133736610413
0.2337709367275238
0.2341209203004837
0.23373390734

0.2338387668132782
0.23396995663642883
0.23344320058822632
0.2333042323589325
0.23376360535621643
0.23368234932422638
0.2336244434118271
0.23382960259914398
0.23382018506526947
0.23329637944698334
0.23351652920246124
0.23333851993083954
0.23366670310497284
0.23400354385375977
0.23354953527450562
0.23378649353981018
0.233443021774292
0.23365069925785065
0.2334287017583847
0.23331120610237122
0.2335757166147232
0.2334638088941574
0.233730286359787
0.23387591540813446
0.23386026918888092
0.2339342087507248
0.23404760658740997
0.23407548666000366
0.23323951661586761
0.23359152674674988
0.23321987688541412
0.23342955112457275
0.23362727463245392
0.2332071214914322
0.23385275900363922
0.2339271605014801
0.23401708900928497
0.23305703699588776
0.2335519641637802
0.2338789403438568
0.23415790498256683
0.23318694531917572
0.23378171026706696
0.2333364337682724
0.23394064605236053
0.23278941214084625
0.23351643979549408
0.23377130925655365
0.23361168801784515
0.23341825604438782
0.23324766755104

0.23381012678146362
0.2339501827955246
0.2340642362833023
0.23329581320285797
0.23346665501594543
0.23369255661964417
0.23316684365272522
0.23405832052230835
0.23417650163173676
0.23334737122058868
0.23338349163532257
0.233488991856575
0.23358212411403656
0.23372595012187958
0.2339007556438446
0.23384782671928406
0.23357151448726654
0.23336204886436462
0.23381954431533813
0.23368580639362335
0.23403076827526093
0.23388519883155823
0.23388415575027466
0.23376710712909698
0.23427022993564606
0.23363007605075836
0.2335251271724701
0.23353014886379242
0.23368078470230103
0.23328526318073273
0.23340506851673126
0.2333516627550125
0.23388774693012238
0.23334605991840363
0.23298650979995728
0.23354780673980713
0.2339324802160263
0.23301458358764648
0.23381873965263367
0.23390331864356995
0.2337304949760437
0.23335759341716766
0.23416902124881744
0.23349709808826447
0.23407670855522156
0.23331663012504578
0.23378527164459229
0.23384183645248413
0.233578160405159
0.2335316687822342
0.2329950630

0.2339140772819519
0.2333698272705078
0.2336006760597229
0.2337987720966339
0.23381023108959198
0.23417654633522034
0.23360514640808105
0.23405182361602783
0.23316386342048645
0.23379908502101898
0.23388874530792236
0.23368898034095764
0.23370783030986786
0.23355276882648468
0.23328901827335358
0.23377029597759247
0.23364901542663574
0.23391878604888916
0.2334941178560257
0.23384615778923035
0.23401343822479248
0.23367315530776978
0.23398751020431519
0.23403716087341309
0.23339377343654633
0.23345567286014557
0.233523890376091
0.23363113403320312
0.23374618589878082
0.2342005968093872
0.23405086994171143
0.2341211587190628
0.23349282145500183
0.23347336053848267
0.23348230123519897
0.23401054739952087
0.23309902846813202
0.23363159596920013
0.23352475464344025
0.23336608707904816
0.23342423141002655
0.233815997838974
0.2339656800031662
0.23357363045215607
0.23377162218093872
0.23382045328617096
0.23334793746471405
0.23342134058475494
0.2336939573287964
0.23389539122581482
0.23343293368

In [8]:
Rmse=sqrt(mean_squared_error(ytest,yhatt))
mae=mean_absolute_error(ytest,yhatt)
R2=r2_score(ytest.T,yhatt.T)
print('RMSE:',Rmse)
print('MAE:',mae)
print('R2:',R2)  

RMSE: 0.4853777657694564
MAE: 0.33249462
R2: 0.9997713882809336
